In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!%cd /content/drive/MyDrive/zeroshot_new_paper

/bin/bash: line 0: fg: no job control


In [3]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

# === KeyZeRoBERToFit v1.0.0 ===

# Dependencies

In [4]:
!pip install transformers
!pip install keybert==0.5.0
!pip install datasets
!pip install flair|
%env PYTORCH_ENABLE_MPS_FALLBACK=1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 s

In [5]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16678 sha256=c8c693e9b7cf780b4081bd0e643e4b74e6d4e3157d8f2ab6996dc0e758a2ab8e
  Stored in directory: /root/.cache/pip/wheels/62/1e/e7/f9ee096e5cc02890a6934a5670ff6e45a3400f330605bd8210
Successfully built pandarallel


In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.0 MB/s eta 0:00:00


In [7]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import evaluation_metrics

def getAgora():
    return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Data Prep

## Source

In [8]:

#which_dataset = 'folhauol'
which_dataset = 'bbc-news'

if which_dataset=='folhauol':
    arq = '/Users/alealcoforado/Documents/Projetos/Datasets/folhauol_clean_df_articles.csv'
    # arq = '/content/drive/MyDrive/folhauol_clean_df_articles.csv'
    raw_data = pd.read_csv(arq)
    raw_data['full_text'] = raw_data['title'].astype(str)+raw_data['text'].astype(str)
#raw_data.head(2)
if which_dataset == 'bbc-news':
  #o dataset bbc-news não tem coluna de título 
  arq = '/content/drive/MyDrive/zeroshot_new_paper/bbc_news_train_plus_test.csv'
  raw_data = pd.read_csv(arq, sep = ';')
  

In [ ]:
raw_data.head(2)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business


In [ ]:
raw_data.Category.value_counts(),raw_data.Category.unique()

(sport            346
 business         336
 politics         274
 entertainment    273
 tech             261
 Name: Category, dtype: int64,
 array(['business', 'tech', 'politics', 'sport', 'entertainment', nan],
       dtype=object))

## Preprocessing

In [ ]:
#acho que não precisamos desta etapa, pois no dataset bbc-news os rótulos
#de categoria possuem apenas uma palavra
#
#categorias = ['poder', 'mercado', 'mundo', 'esporte', 'tec',
#        'ambiente', 'equilibrioesaude',  'educacao', 'tv', 
#       'ciencia',  'turismo','comida']
#raw_data = raw_data[raw_data.category.isin(categorias)].copy()
# di_cat = {'poder':'poder e política',
#           'mercado': 'mercado',
#           'mundo':'notícias de países',
#           'esporte':'esportes',
#           'tec':'tecnologia',
#         'ambiente':'ambiente',
#          'equilibrioesaude':'equilíbrio e saúde',
#           'educacao': 'educação',
#           'tv':'tv, televisão e entretenimento', 
#        'ciencia': 'ciência',
#          'turismo': 'turismo',
#           'comida':'comida'}
# dict_classes_folha = { 'poder':"Poder e Política no Brasil",
# 'mercado':"Mercado",
# 'mundo':"Notícias de fora do Brasil",
# 'esporte':"Esporte",
# 'tec':"Tecnologia",
# 'ambiente':"Meio Ambiente",
# 'equilibrioesaude':"Equilíbrio e Saúde",
# 'educacao':"Educação",
# 'tv':"TV, Televisão e Entretenimento",
# 'ciencia':"Ciência",
# 'turismo':"Turismo",
# 'comida':"Comida" }
# raw_data['label_name'] = raw_data.category.map(dict_classes_folha)


In [ ]:
# raw_data['label_name'].unique()

array(['Poder e Política no Brasil', 'Mercado',
       'Notícias de fora do Brasil', 'Esporte', 'Tecnologia',
       'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação',
       'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'],
      dtype=object)

## Dataset build

### Train Data

In [9]:
arq_train = '/content/drive/MyDrive/zeroshot_new_paper/BBC News Train.csv'
train = pd.read_csv(arq_train)
# train = raw_data[~(raw_data['full_text'].isnull())].copy()
# train = raw_data[~(raw_data['full_text'].isnull())].sample(max_inferences,random_state=42).copy()

In [10]:
labels_list = list(train['Category'].unique())

In [ ]:
train.head(2)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business


# Sentence Extraction

In [11]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
len(train)

1490

In [12]:
try:
    print(train['sentences'])
except:
    train['sentences'] = train['Text'].progress_apply(lambda txt:tokenizer.tokenize(txt))
    train.head(2)

100%|██████████| 1490/1490 [00:01<00:00, 1309.05it/s]


In [13]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
train[['len_sentences']].describe()

100%|██████████| 1490/1490 [00:00<00:00, 330207.81it/s]


,len_sentences
count,1490.000000
mean,17.587248
std,10.017686
min,4.000000
25%,11.000000
50%,15.000000
75%,21.000000
max,139.000000


# Zero-Shot

## Data

In [14]:
# zeroshot_method = "run_all"     #### not implemented yet
max_inferences = len(train)#estava em 5000

zeroshot_method = "probability_threshold"
probability_goal = 0.30

# zeroshot_method = "top_n_goal"  #### not implemented yet
top_n_goal = 4

zeroshot_config = {
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': "this article is about {}.",#O tema principal deste texto é {}.
    'random_state':422
}
zeroshot_config

{'method': 'probability_threshold',
 'prob_goal': 0.3,
 'top_n_goal': 4,
 'max_inferences': 1490,
 'labels': ['business', 'tech', 'politics', 'sport', 'entertainment'],
 'template': 'this article is about {}.',
 'random_state': 422}

In [15]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)

1490

In [16]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

1490

## Model

In [17]:
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli",device="cpu")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
def runZeroShotPipeline(classifier,data,config):
   # labels =  list(dict_classes_folha.values())
   goal_count = dict(zip(config['labels'],np.zeros(len(config['labels']))))
   print("# data:",len(data))
   print(config)
   preds = []
   t0 = time.time()
   for text in data:
      pred = classifier(text, candidate_labels=config['labels'], 
                           hypothesis_template=config['template'], multi_label=False)
      preds.append(pred)

      top_prob = pred['scores'][0]
      # print(top_prob)
      top_label = pred['labels'][0]
      # print(top_label)
      if (top_prob>=config['prob_goal']):
         goal_count[top_label] += 1
      if len(preds) % 50 == 0:
         print("Preds:",len(preds)," - Total time:",round(time.time()-t0,2),"seconds")
         print(goal_count)
      if all(count >= config['top_n_goal'] for count in list(goal_count.values())):
         break  ### stop loop if goal is met
   print(len(preds))
   return preds

## Inference

In [19]:
%%time
zeroshot_results = runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)
len(zeroshot_results)

# data: 1490
{'method': 'probability_threshold', 'prob_goal': 0.3, 'top_n_goal': 4, 'max_inferences': 1490, 'labels': ['business', 'tech', 'politics', 'sport', 'entertainment'], 'template': 'this article is about {}.', 'random_state': 422}
42
CPU times: user 2min 37s, sys: 0 ns, total: 2min 37s
Wall time: 2min 38s


42

In [ ]:
### old
# %%time

# labels =  list(dict_classes_folha.values())
# template = "O tema principal deste texto é {}."

# def zero_shot_classifier(row):
#     return classifier(row['sentence_to_zeroshot'], candidate_labels=labels, hypothesis_template=template, multi_label=False)

# zeroshot_results = zeroshot_data[0:2].progress_apply(zero_shot_classifier,axis=1)

In [20]:
def formatZeroShotResults(results):
  df_results = pd.DataFrame(pd.Series(results).to_dict())
  label_probabilities = list(map(mapper,enumerate(results)))
  label_probabilities_df = pd.DataFrame(label_probabilities,index=df_results.columns)
  
  return label_probabilities_df

def mapper(item_tuple):
  index, item = item_tuple
  return dict(zip(item["labels"], item["scores"]))

label_probabilities_df = formatZeroShotResults(zeroshot_results)

In [21]:
def get_top_label(row):
  return row.idxmax()
def get_top_prob(row):
  return row.max()

label_results = label_probabilities_df.apply(get_top_label,axis=1)
prob_results = label_probabilities_df.apply(get_top_prob,axis=1)
label_results_df = pd.Series(label_results,name='prediction')
label_results_df.head()

0        tech
1       sport
2    politics
3    business
4    business
Name: prediction, dtype: object

In [23]:
true_labels_df = raw_data[raw_data.index.isin(train.index)].Category
# true_labels = raw_data[raw_data.index.isin(train.index)].Category
# true_labels_df = pd.Series(true_labels).map(dict_classes_folha)

In [24]:
true_labels_df

0            business
1            business
2            business
3                tech
4            business
            ...      
1485    entertainment
1486    entertainment
1487         business
1488             tech
1489             tech
Name: Category, Length: 1490, dtype: object

In [26]:
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)
vector_true = final_result_df['Category'].str.strip().str.lower() == final_result_df['prediction'].str.strip().str.lower()

## Metrics

In [27]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction','Category'])

In [29]:
evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'].to_list(),final_result_df_encoded['Category_code'].to_list())

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.00738255033557047},
  {'precision': 0.274679682733374},
  {'recall': 0.00738255033557047},
  {'f1': 0.014328302817197028}],
 'macro': [{'accuracy': 0.00738255033557047},
  {'precision': 0.22258297258297258},
  {'recall': 0.005756143905187157},
  {'f1': 0.011180855256961017}]}

### Probabilities

In [30]:
df_predictions_probs = pd.concat([final_result_df_encoded,pd.Series(prob_results,name='top_probability'),pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [31]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.333}
{'accuracy in top 2': 0.25}
{'accuracy in top 3': 0.222}
{'accuracy in top 4': 0.167}
{'accuracy in top 5': 0.138}
{'accuracy in top 6': 0.176}
{'accuracy in top 7': 0.158}
{'accuracy in top 8': 0.171}
{'accuracy in top 9': 0.182}
{'accuracy in top 10': 0.17}
{'accuracy in top 11': 0.2}
{'accuracy in top 12': 0.212}
{'accuracy in top 13': 0.204}
{'accuracy in top 14': 0.196}
{'accuracy in top 15': 0.193}
{'accuracy in top 16': 0.19}
{'accuracy in top 42': 0.131}


In [32]:
agora = getAgora()
output_path_results = '/content/drive/MyDrive/zeroshot_new_paper/predictions_and_probabilities_test_{}.csv'.format(agora)
output_path_configs = '/content/drive/MyDrive/zeroshot_new_paper/zeroshot_config_test_{}.csv'.format(agora)

df_predictions_probs.to_csv(output_path_results)
pd.DataFrame(zeroshot_config).to_csv(output_path_configs)
print(output_path_results,output_path_configs)

/content/drive/MyDrive/zeroshot_new_paper/predictions_and_probabilities_test_2023_02_09__21_17_02.csv /content/drive/MyDrive/zeroshot_new_paper/zeroshot_config_test_2023_02_09__21_17_02.csv


# Load Previous Results

In [ ]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [ ]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)